In [4]:
import os
import pandas as pd
import string
import re
import mysql.connector as mysql
from mysql.connector import Error

In [19]:
class database:
    def __init__(self, dbName: str, df: pd.DataFrame, table_name: str):
        self.dbName = dbName
        self.df = df
        self.table_name = table_name

    def DBConnect(self):
        conn = mysql.connect(host='localhost', user='root', password='',
                            database=self.dbName, buffered=True)
        cur = conn.cursor()
        return conn, cur

    def emojiDB(self) -> None:
        conn, cur = self.DBConnect()
        dbQuery = f"ALTER DATABASE {self.dbName} CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;"
        cur.execute(dbQuery)
        conn.commit()

    def createDB(self) -> None:
        conn, cur = self.DBConnect()
        cur.execute(f"CREATE DATABASE IF NOT EXISTS {self.dbName};")
        conn.commit()
        cur.close()

    def createTables(self) -> None:
        conn, cur = self.DBConnect()
        sqlFile = 'D:\work\data\day5_schema.sql'
        fd = open(sqlFile, 'r')
        readSqlFile = fd.read()
        fd.close()

        sqlCommands = readSqlFile.split(';')

        for command in sqlCommands:
            try:
                res = cur.execute(command)
            except Exception as ex:
                print("Command skipped: ", command)
                print(ex)
        conn.commit()
        cur.close()

        return

    def preprocess_df(self, df) -> pd.DataFrame:

        df.rename({'original_text':'clean_text'}, axis=1, inplace=True) # rename 'original_text' with 'clean_text'
        df = df.drop(columns=['possibly_sensitive'], axis=1)
        # Remove hyperlinks
        rgx = lambda x: re.sub('http[s]?', '', x)
        df['clean_text'] = df['clean_text'].map(rgx)

        # Remove punctuation
        df['clean_text']= \
        df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))

        df['polarity'] = pd.to_numeric(df['polarity'],errors='coerce')  # change polarity to numeric
        df = df.dropna() # remove rows and columns with Null/NaN values.
        return df


    def insert_to_tweet_table(self) -> None:
            
        conn, cur = self.DBConnect()

        self.df = self.preprocess_df(self.df)
        
        for _, row in self.df.iterrows():
            sqlQuery = f"""INSERT INTO {self.table_name} (created_at, source, clean_text, polarity, subjectivity,lang, favorite_count, retweet_count, original_author, followers_count, friends_count, hashtags, user_mentions, place)
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
            data = (row[0], row[1], row[2], row[3], (row[4]), (row[5]), row[6], row[7], row[8], row[9], row[10], row[11],
                    row[12], row[13])
            try:
                # Execute the SQL command
                cur.execute(sqlQuery, data)
                # Commit your changes in the database
                conn.commit()
                print("Data Inserted Successfully")
            except Exception as e:
                conn.rollback()
                print("Error: ", e)
        return

    def db_execute_fetch(self, *args, many=False, tablename='', rdf=True, **kwargs) -> pd.DataFrame:
        connection, cursor1 = self.DBConnect(**kwargs)
        if many:
            cursor1.executemany(*args)
        else:
            cursor1.execute(*args)

        # get column names
        field_names = [i[0] for i in cursor1.description]

        # get column values
        res = cursor1.fetchall()

        # get row count and show info
        nrow = cursor1.rowcount
        if tablename:
            print(f"{nrow} recrods fetched from {tablename} table")

        cursor1.close()
        connection.close()

        # return result
        if rdf:
            return pd.DataFrame(res, columns=field_names)
        else:
            return res


In [20]:
if __name__ == "__main__":
    df = pd.read_csv('D:\work\data\processed_tweetdata.csv')
    db = database(dbName='itweets', df=df, table_name='tweet')
    db.createDB()
    db.emojiDB()
    db.createTables()
    db.insert_to_tweet_table()

Command skipped:  

1065 (42000): Query was empty
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'field list'
Error:  1054 (42S22): Unknown column 'lang' in 'fi

In [7]:
!pip install streamlit

In [8]:
!pip install plotly

In [9]:
import streamlit as st
import altair as alt
from wordcloud import WordCloud
import plotly.express as px

In [10]:
st.set_page_config(page_title="Day 5: Topic modeling and Sentiment Analysis ", layout="wide")

In [11]:
"""def loadData():
    query = "select * from TweetInformation"
    df = db_execute_fetch(query, dbName="tweets", rdf=True)
    return df
"""


'def loadData():\n    query = "select * from TweetInformation"\n    df = db_execute_fetch(query, dbName="tweets", rdf=True)\n    return df\n'

In [12]:
def sql(query, cursor):
    '''
    Takes an SQL query string, and outputs a
    dataframe representation of the query result.
    '''
    # Execute the sql query
    cursor.execute(query)

    # Get the query into a dataframe and set columns
    df = pd.DataFrame(cursor.fetchall())
    df.columns = [x[0] for x in cursor.description]

    # Set the sql id as the dataframe index
    index_column = df.columns[0]
    df.set_index(index_column, drop=True, inplace=True)

    return df

In [13]:
def text_category(p):
    """
    A function  that takes a value p and returns, depending on the value of p, 
    a string 'positive', 'negative' or 'neutral'
    """
    if p > 0 : return 'positive'
    elif p == 0: return 'neutral'
    return 'negative'

In [14]:
# display the db based on the polarity
def display_df_polarity(p):
    df = loadData()
    df['score'] = df['polarity'].apply(text_category)
    if p == 'positive':
        st.write(df[df['score'] == 'positive'])
    elif p == 'negative':
        st.write(df[df['score'] == 'negative'])
    elif p == 'neutral':
        st.write(df[df['score'] == 'neutral'])
    else:
        st.write(df)

In [15]:
# Count the number of positive, neutral, and negative
def polarity_count():
    df = loadData()
    df['score'] = df['polarity'].apply(text_category) 
    sc = list(df['score'])
    return { 'positive': sc.count('positive'), 'neutral': sc.count('neutral'),
                            'negative': sc.count('negative')  } 


In [16]:
def selectHashTag():
    df = loadData()
    hashTags = st.multiselect("choose combaniation of hashtags", list(df['hashtags'].unique()))
    if hashTags:
        df = df[np.isin(df, hashTags).any(axis=1)]
        st.write(df)

In [17]:
def selectLocAndAuth():
    df = loadData()
    location = st.multiselect("choose Location of tweets", list(df['place_coordinate'].unique()))
    lang = st.multiselect("choose Language of tweets", list(df['language'].unique()))

    if location and not lang:
        df = df[np.isin(df, location).any(axis=1)]
        st.write(df)
    elif lang and not location:
        df = df[np.isin(df, lang).any(axis=1)]
        st.write(df)
    elif lang and location:
        location.extend(lang)
        df = df[np.isin(df, location).any(axis=1)]
        st.write(df)
    else:
        st.write(df)

In [18]:
polarity = st.selectbox('choose polarity of tweets', ('All', 'positive', 'negative', 'neutral'))
display_df_polarity(polarity)

2021-06-30 14:54:49.718 
  command:

    streamlit run C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


NameError: name 'loadData' is not defined

In [ ]:
def barChart(data, title, X, Y):
    title = title.title()
    st.title(f'{title} Chart')
    msgChart = (alt.Chart(data).mark_bar().encode(alt.X(f"{X}:N", sort=alt.EncodingSortField(field=f"{Y}", op="values",
                order='ascending')), y=f"{Y}:Q"))
    st.altair_chart(msgChart, use_container_width=True)

In [ ]:
def barChart():
    st.title('Bar Chart')
    count = polarity_count()
    data = pd.DataFrame({
    'Sentiment': list(count.keys()),
    'Tweets': [count[key] for key in count.keys()],
                })
    bar_fig = px.bar(data, x='Sentiment', y='Tweets')
    st.plotly_chart(bar_fig)

In [ ]:
def wordCloud():
    df = loadData()
    cleanText = ''
    for text in df['clean_text']:
        tokens = str(text).lower().split()

        cleanText += " ".join(tokens) + " "

    wc = WordCloud(width=650, height=450, background_color='white', min_font_size=5).generate(cleanText)
    st.title("Tweet Text Word Cloud")
    st.image(wc.to_array())

In [ ]:
# topic modeling
def wordCloud():
    df = loadData()
    # Convert to lowercase
    df['clean_text'] = df['clean_text'].map(lambda x: x.lower())
    # Join the different processed titles together.
    long_string = ','.join(list(df['clean_text'].values))

    # Create a WordCloud object
    wordcloud = WordCloud(background_color="black", width=650, height=450, \
                             min_font_size=5, contour_color='steelblue')
    # Generate a word cloud
    wordcloud.generate(long_string)
    st.title("Tweet Text Word Cloud")
    st.image(wordcloud.to_array())

In [ ]:
def stBarChart():
    df = loadData()
    dfCount = pd.DataFrame({'Tweet_count': df.groupby(['original_author'])['clean_text'].count()}).reset_index()
    dfCount["original_author"] = dfCount["original_author"].astype(str)
    dfCount = dfCount.sort_values("Tweet_count", ascending=False)

    num = st.slider("Select number of Rankings", 0, 50, 5)
    title = f"Top {num} Ranking By Number of tweets"
    barChart(dfCount.head(num), title, "original_author", "Tweet_count")

In [ ]:
def langPie():
    df = loadData()
    dfLangCount = pd.DataFrame({'Tweet_count': df.groupby(['language'])['clean_text'].count()}).reset_index()
    dfLangCount["language"] = dfLangCount["language"].astype(str)
    dfLangCount = dfLangCount.sort_values("Tweet_count", ascending=False)
    dfLangCount.loc[dfLangCount['Tweet_count'] < 10, 'lang'] = 'Other languages'
    st.title(" Tweets Language pie chart")
    fig = px.pie(dfLangCount, values='Tweet_count', names='language', width=500, height=350)
    fig.update_traces(textposition='inside', textinfo='percent+label')

    colB1, colB2 = st.beta_columns([2.5, 1])

    with colB1:
        st.plotly_chart(fig)
    with colB2:
        st.write(dfLangCount)

In [ ]:
# draws pie chart of the polarity
def pieChart():
    st.title('Pie Chart')
    count = polarity_count()
    pie_fig = px.pie(values=[count[key] for key in count.keys()], names=list(count.keys()))
    st.plotly_chart(pie_fig)

In [ ]:
st.markdown("***")
st.markdown("***")
st.title("Data Visualizations")
random_tweet = st.selectbox('Visualizations', 
                ('Topic Modeling','Bar Chart','Pie Chart'))
if random_tweet == 'Topic Modeling':
    wordCloud()
elif random_tweet == 'Bar Chart':
    barChart()
elif random_tweet == 'Pie Chart':
    pieChart()

In [ ]:
st.title("Data Display")
selectHashTag()
st.markdown("<p style='padding:10px; background-color:#000000;color:#00ECB9;font-size:16px;border-radius:10px;'>Section Break</p>", unsafe_allow_html=True)
selectLocAndAuth()
st.title("Data Visualizations")
wordCloud()
with st.beta_expander("Show More Graphs"):
    stBarChart()
    langPie()

In [ ]:
#kkkk